In [1]:
# default_exp core

In [20]:
#|export
from typing import List, Union, Dict
from datetime import date, datetime
from pydantic import NonNegativeInt, NonNegativeFloat
from pydantic.dataclasses import dataclass
import io

In [25]:
# Tipos de campos

# Tipos de registro

Campo = Union[Fecha, Cadena, Entero, EnteroSigno, Flotante, FlotanteSigno]


@dataclass
class Registro:
    campos: List[Campo]
    largo: int
        
    def __str__(self):
        return ''.join([str(v) for v in self.campos])
    
    def __post_init__(self):
        largo_campos = sum([len(v) for v in self.campos])
        if abs(self.largo) != largo_campos:
            raise ValueError(f"La suma del largo de los campos ({largo_campos}) no es igual al largo del registro ({self.largo}).")
            
            
            
def mk_reg(campos: List[Campo], largo: int):
    return Registro(**{'campos': campos, 'largo': largo})


# Definir un archivo (como el R07)


@dataclass
class Archivo:
    registros: List[Registro]
    nombre: str
    fecha: date
        
    def escribir(self, ruta: str) -> Union[str, None]:
        try:
            with open(f'{ruta}{self.fecha.isoformat().replace("-", "")}_{self.nombre}', 'w') as archivo:
                for reg in self.registros:
                    archivo.write(f'{str(reg)}\n')
        except Exception as e:
            return 'Se produjo el error: ' + str(e)
        
    def escribir_en_memoria(self):
        try:
            s = io.StringIO()
            for reg in self.registros:
                s.write('{}\n'.format(str(reg)))

            return s

        except Exception as e:
            return 'Se produjo el error: ' + str(e)


In [26]:
#hide
from nbdev.showdoc import *

# Core

> API details.

## Tipos de Campo

In [3]:
#|export
@dataclass
class Fecha:
    fecha: date
    
    def __str__(self):
        return self.fecha.isoformat().replace("-", "")
    
    def __len__(self):
        return 8

In [5]:
#|export
def fecha(fecha: date) -> Fecha:
    return Fecha(fecha=fecha)

In [9]:
#|export
@dataclass
class Cadena:
    valor: str
    largo: NonNegativeInt
    
    def __str__(self):
        return self.valor[0: min(len(self.valor), abs(self.largo))].ljust(abs(self.largo), ' ')
    
    def __len__(self):
        return abs(self.largo)

In [11]:
#|export
def cadena(valor: str, largo: NonNegativeInt) -> Cadena:
    return Cadena(valor=valor, largo=largo)

In [13]:
#|export
@dataclass
class Entero:
    valor: NonNegativeInt
    largo: NonNegativeInt
        
    def __str__(self):
        return str(abs(int(round(self.valor, 0)))).zfill(abs(self.largo))
    
    def __len__(self):
        return abs(self.largo)

In [27]:
#|export    
def entero(valor: NonNegativeInt, largo: NonNegativeInt) -> Entero:
    return Entero(valor=valor, largo=largo)

In [15]:
#|export
@dataclass
class EnteroSigno:
    valor: int
    largo: NonNegativeInt
        
    def __str__(self):
        sgn = "+" if self.valor >= 0 else "-"
        return str(abs(int(round(self.valor, 0)))).zfill(abs(self.largo)) + sgn
    
    def __len__(self):
        return abs(self.largo) + 1

In [16]:
#|export    
def entero_signo(valor: int, largo: NonNegativeInt) -> EnteroSigno:
    return EnteroSigno(valor=valor, largo=largo)

In [21]:
#|export
@dataclass
class Flotante:
    valor: NonNegativeFloat
    largo_entero: NonNegativeInt
    largo_decimales: NonNegativeInt

    def __str__(self):
        entero = int(abs(self.valor))
        decimales = round(abs(self.valor) - entero, abs(self.largo_decimales))
        return (
            str(entero).rjust(abs(self.largo_entero), '0') +
            str(decimales).replace("0.", "").ljust(abs(self.largo_decimales), '0')
        )
    
    def __len__(self):
        return abs(self.largo_entero) + abs(self.largo_decimales)

In [22]:
#|export  
def flotante(
    valor: NonNegativeFloat, 
    largo_entero: NonNegativeInt, 
    largo_decimales: NonNegativeInt
) -> Flotante:
    return Flotante(
        valor=valor,
        largo_entero=largo_entero,
        largo_decimales=largo_decimales
    )

In [23]:
#|export  
@dataclass
class FlotanteSigno:
    valor: float
    largo_entero: NonNegativeInt
    largo_decimales: NonNegativeInt

    def __str__(self):
        sgn = "+" if self.valor >=0 else "-"
        entero = int(abs(self.valor))
        decimales = round(abs(self.valor) - entero, abs(self.largo_decimales))
        return (
            str(entero).rjust(abs(self.largo_entero), '0') +
            str(decimales).replace("0.", "").ljust(abs(self.largo_decimales), '0') +
            sgn
        )
    
    def __len__(self):
        return abs(self.largo_entero) + abs(self.largo_decimales) + 1

In [28]:
#|export  
def flotante_signo(
    valor: float, 
    largo_entero: NonNegativeInt, 
    largo_decimales: NonNegativeInt
) -> FlotanteSigno:
    return FlotanteSigno(
        valor=valor,
        largo_entero=largo_entero,
        largo_decimales=largo_decimales,
    )